# Backfill Lear User Data from Auth
This notebook updates entries in the `users` table in lear db where first/last name are missing with data from the auth api.

## Setup environment
Before running following environment setup snippets, ensure environment variables found in `default-bcr-business-setup-TEST` notebook contain the correct values.

In [ ]:
%run /workspaces/lear/tests/data/default-bcr-business-setup-TEST.ipynb

In [ ]:
%run /workspaces/lear/tests/data/common/legal_api_utils.ipynb
%run /workspaces/lear/tests/data/common/auth_api_utils.ipynb

In [ ]:
import json
from sqlalchemy import and_, or_
from legal_api.models import User

## Input Values for filing
Please update following values to appropriate values before running subsequent code snippets!!

In [ ]:
verify_ssl=True # set to False if using proxy to debug requests

## Tokens

In [ ]:
auth_token = get_auth_token(verify_ssl)
assert auth_token
# auth_token

# Update Lear Users

In [ ]:
# get lear users with no first/last names and has a username
query = db.session.query(User) \
    .filter(
        or_(
            User.firstname == None,
            User.firstname == ''
        ),
        or_(
            User.lastname == None,
            User.lastname == ''
        ),
        User.username != None,
        User.username != ''
    )

lear_users = query.all()
assert(lear_users)
lear_users_count = len(lear_users)
print(f'lear users to process: {lear_users_count}')

In [ ]:
# loop through lear users with missing info and make call to auth users endpoint.  if valid response comes back, populate lear user
# with missing info(first name, last name, email) where possible
for lear_user in lear_users:

    try:
        user_name = lear_user.username
        r = get_user_by_username(auth_token, user_name, verify_ssl)

        if r.status_code != 200:
            raise Exception(f'error {r.status_code}, {r.text}')

        auth_user_dict = json.loads(r.text)
        first_name = auth_user_dict.get('firstname')
        last_name = auth_user_dict.get('lastname')
        email = auth_user_dict.get('email')
        if first_name or last_name or email:
            lear_user.firstname = first_name
            lear_user.lastname = last_name
            lear_user.email = email
            lear_user.save()
        else:
            raise Exception('no first name or last name provided by auth response')

        print(f'lear user user_name: {user_name} updated succeeded')
    except Exception as err:
        print(f'lear user user_name: {user_name} updated failed, {err}')
